In [ ]:
from Crypto.Cipher import Blowfish, PKCS1_OAEP, AES
from Crypto.PublicKey import RSA
from Crypto.Util.Padding import pad, unpad
from binascii import hexlify , unhexlify
import hashlib , json, string, random
from stegano import lsb
from datetime import datetime

# Key Generator
def key_generator(size, case="default", punctuations="required"):
    if case=="default" and punctuations=="required":
        return ''.join(random.choices(string.ascii_uppercase + string.ascii_lowercase + string.digits + string.punctuation, k = size))
    elif case=="upper-case-only" and punctuations=="required":
        return ''.join(random.choices(string.ascii_uppercase + string.digits + string.punctuation, k = size))
    elif case=="lower-case-only"  and punctuations=="required":
        return ''.join(random.choices(string.ascii_lowercase + string.digits + string.punctuation, k = size))
    elif case=="default" and punctuations=="none":
        return ''.join(random.choices(string.ascii_uppercase + string.digits + string.ascii_lowercase, k = size))
    elif case=="lower-case-only"  and punctuations=="none":
        return ''.join(random.choices(string.ascii_lowercase + string.digits , k = size))
    elif case=="upper-case-only" and punctuations=="none":
        return ''.join(random.choices(string.ascii_uppercase + string.digits, k = size))

# Plaintext Input
plaintext = input('Enter Plaintext: ').encode()

log_plaintext_length = len(hexlify(plaintext))

# Password for Keys
password = input('Enter Password: ')
log_password_length = len(password)

log_start_time = datetime.now()

hash = hashlib.sha1()
hash.update(password.encode())
password_encryption_cipher = AES.new( hash.hexdigest()[:16].encode() , AES.MODE_CBC, iv= '16bitAESInitVect'.encode())

# Dictionary of Keys
keys_iv = {}

# Blowfish Layer 1

blowfish_key =  key_generator(size=16).encode()
blowfish_cipher = Blowfish.new(blowfish_key, Blowfish.MODE_CBC)

blowfish_ciphertext = blowfish_cipher.encrypt(pad(plaintext, Blowfish.block_size ))

keys_iv['blowfish_iv'] = hexlify(blowfish_cipher.iv).decode()
keys_iv['blowfish_key'] = hexlify(blowfish_key).decode()

# RSA Layer 2

rsa_key = RSA.generate(2048)
rsa_private_key = rsa_key
rsa_public_key = rsa_key.publickey()

cipher_rsa = PKCS1_OAEP.new(rsa_public_key)
rsa_plaintext = blowfish_ciphertext

rsa_ciphertext = bytearray()
for i in range(0, len(rsa_plaintext), 190):
    rsa_ciphertext.extend(cipher_rsa.encrypt(rsa_plaintext[i:i+190]))

keys_iv['rsa_n'] = rsa_private_key.n
keys_iv['rsa_e'] = rsa_private_key.e
keys_iv['rsa_d'] = rsa_private_key.d

# AES Layer 3
aes_key =  key_generator(size=16).encode()
aes_cipher = AES.new(aes_key, AES.MODE_CBC)
aes_plaintext = rsa_ciphertext

aes_ciphertext = aes_cipher.encrypt(pad(aes_plaintext, AES.block_size))

ciphertext = aes_ciphertext

log_ciphertext_length = len(hexlify(ciphertext))

keys_iv['aes_iv'] = hexlify(aes_cipher.iv).decode()
keys_iv['aes_key'] = hexlify(aes_key).decode()


# Encryption of Key and IV String
encrypted_keys_and_iv = hexlify(password_encryption_cipher.encrypt(pad(json.dumps(keys_iv).encode(), AES.block_size)))

#LSB Steg
lsb_stegano_image = lsb.hide("/content/drive/MyDrive/SEM-6/CDSS/test/cover_image.png", encrypted_keys_and_iv.decode())
lsb_stegano_image.save("/content/drive/MyDrive/SEM-6/CDSS/test/stego_image.png")

log_end_time = datetime.now()

log_duration = str(log_end_time - log_start_time)

with open('/content/drive/MyDrive/SEM-6/CDSS/logs/encryption-log.txt', 'a+') as log_file:
    log_file.write( "\n| "   +str(log_plaintext_length)
                   +"          | "+str(log_ciphertext_length)
                   +"          | "+str(log_password_length)
                   +"         | "+log_start_time.strftime("%H:%M:%S")
                   +"   | "+log_end_time.strftime("%H:%M:%S")
                   +"  | "+str(log_duration)
                   +" |"
                  )

print('Plaintext in Hexadecimal: ')
print(hexlify(plaintext).decode())
print('Ciphertext: ')
print(hexlify(ciphertext).decode())

print('Encryption Complete!')

print('Encryption Metrics:')

print( "Length of Plaintext   :  "+str(log_plaintext_length)+
       "\nLength of Ciphertext  :  "+str(log_ciphertext_length)+
       "\nLength of Password    :  "+str(log_password_length)+
       "\nEncryption Time (sec) :  "+str(log_duration))



In [ ]:
# Password for Keys
password = input('Enter Password: ')

log_password_length = len(password)
log_ciphertext_length = len(hexlify(ciphertext))

log_start_time = datetime.now()

# LSB Steg
unhide_encrypted_keys_and_iv = lsb.reveal("/content/drive/MyDrive/SEM-6/CDSS/test/stego_image.png").encode()

hash = hashlib.sha1()
hash.update(password.encode())
password_decryption_cipher = AES.new( hash.hexdigest()[:16].encode() , AES.MODE_CBC, iv= '16bitAESInitVect'.encode())

decrypted_keys_iv = json.loads(unpad(password_decryption_cipher.decrypt(unhexlify(unhide_encrypted_keys_and_iv)), AES.block_size))

#Initializations
decryption_key_aes = unhexlify(decrypted_keys_iv['aes_key'])
decryption_iv_aes = unhexlify(decrypted_keys_iv['aes_iv'])
decryption_key_rsa = RSA.construct(rsa_components = (decrypted_keys_iv['rsa_n'] , decrypted_keys_iv['rsa_e'] , decrypted_keys_iv['rsa_d']))
decryption_iv_blowfish = unhexlify(decrypted_keys_iv['blowfish_iv'])
decryption_key_blowfish = unhexlify(decrypted_keys_iv['blowfish_key'])


aes_cipher_decryption = AES.new(decryption_key_aes, AES.MODE_CBC, iv=decryption_iv_aes)
rsa_cipher_decryption = PKCS1_OAEP.new(decryption_key_rsa)
blowfish_cipher_decryption = Blowfish.new(decryption_key_blowfish, Blowfish.MODE_CBC, iv=decryption_iv_blowfish)

# AES DECRYPTION
ciphertext_rsa = unpad(aes_cipher_decryption.decrypt(ciphertext), AES.block_size)
# RSA DECRYPTION
ciphertext_blowfish = bytearray()
for i in range(0, len(ciphertext_rsa),256):
    ciphertext_rsa_segment = ciphertext_rsa[i:i+256]
    ciphertext_blowfish.extend(rsa_cipher_decryption.decrypt(ciphertext_rsa_segment))

# BLOWFISH DECRYPTION
decrypted_plaintext = unpad(blowfish_cipher_decryption.decrypt(ciphertext_blowfish), Blowfish.block_size)

log_end_time = datetime.now()
log_duration = str(log_end_time - log_start_time)
log_plaintext_length = len(hexlify(decrypted_plaintext))

with open('/content/drive/MyDrive/SEM-6/CDSS/logs/decryption-log.txt', 'a+') as log_file:
    log_file.write( "\n| "   +str(log_ciphertext_length)
                   +"          | "+str(log_plaintext_length)
                   +"          | "+str(log_password_length)
                   +"         | "+log_start_time.strftime("%H:%M:%S")
                   +"   | "+log_end_time.strftime("%H:%M:%S")
                   +"  | "+str(log_duration)
                   +" |"
                  )

print("Decrypted Ciphertext: ", decrypted_plaintext.decode())
print('File Decryption Complete!')

Enter Password: df
Decrypted Ciphertext:  yftgyh
File Decryption Complete!


In [2]:
!pip install pycryptodome



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 31.1 MB/s eta 0:00:00


In [3]:
!pip install stegano

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 48.4 MB/s eta 0:00:00
  Attempting uninstall: pillow
    Found existing installation: pillow 11.2.1
    Uninstalling pillow-11.2.1:
      Successfully uninstalled pillow-11.2.1
